# Human Machine Agreement

In [2]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


In [3]:
human = pd.read_csv("/home/ada/humor/data/stand_up_dataset/human_ans - Sheet1.csv")
models = pd.read_csv("/home/ada/humor/data/stand_up_dataset/llm_ans - Sheet1.csv")

## Percentage Agreement

In [20]:
human['human'] = (human['funny'] >= 5).astype(int)
merged_df = pd.merge(human[['comedian', 'sentence', 'human']], models, on=['comedian', 'sentence'])

model_names = ['gemma', 'gemma2', 'llama', 'phi', 'chatgpt', 'claude']  

agreement_rates = []
for model in model_names:
    matches = (merged_df['human'] == merged_df[model])
    agreement_rate = matches.mean() * 100
    agreement_rates.append({'model': model, 'agreement rate': agreement_rate.item()})

results_df = pd.DataFrame(agreement_rates)
average_rate = results_df['agreement rate'].mean()
results_df.loc[len(results_df.index)] = ["Overall", average_rate]
results_df 

,model,agreement rate
0,gemma,68.789809
1,gemma2,68.789809
2,llama,61.146497
3,phi,66.878981
4,chatgpt,28.662420
5,claude,64.968153
6,Overall,59.872611


# Graph percentage agreement?

In [30]:
gemma = results_df.loc[results_df['model'] == 'gemma'].assign(model="gemma")
phi = results_df.loc[results_df['model'] == 'phi'].assign(model="phi")
gemma2 = results_df.loc[results_df['model'] == 'gemma2'].assign(model="gemma2")
llama = results_df.loc[results_df['model'] == 'llama'].assign(model="llama")
chatgpt = results_df.loc[results_df['model'] == 'chatgpt'].assign(model="chatgpt")
claude = results_df.loc[results_df['model'] == 'claude'].assign(model="claude")

df = pd.concat([gemma, phi, gemma2, llama, chatgpt, claude]).reset_index(drop=True)
plot = sns.displot(df, x="agreement_rate", hue="model", kind="hist", shrink=1, multiple="dodge")
plot.fig.suptitle("Scores Across Various Models", fontsize=16, y=1.05)  
plot.set_axis_labels("Score", "Count")  

plot.savefig("experiment_02.png")
plt.show()

ValueError: Could not interpret value `agreement_rate` for `x`. An entry with this name does not appear in `data`.